In [1]:
#1.从defillama上面抓取 latest 24h TVL, trade volume, logo, name, yield 
#把最新数据整合到一个 data frame 里面

In [2]:
import requests
from web3 import Web3
from bs4 import BeautifulSoup
import pandas as pd
import json
from sklearn.neighbors import LocalOutlierFactor
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
from scipy import stats

In [3]:
protocols = ["gmx", "frax-finance", "convex-finance", "curve-finance", "synthetix", 'dYdX', 'gains-network', 'umami-finance', 'LooksRare', 'Beefy', 'ribbon', 'redacted-protocol', 'dopex', 'kujira-protocol', 'polynomial-protocol','Lido', 'MakerDAO', 'aave', 'JustLend', 'uniswap', 'compound-finance', 'pancakeswap', 'venus', 'pendle', 'morpho']
protocols_pools = ['frax', 'gmx-v1', 'gmx-v2', 'curve-dex', "synthetix", 'dydx', 'looksrare', 'beefy', "convex-finance",'gains-network','umami-finance', 'ribbon','makerdao','lido','aave-v3','pancakeswap-amm','venus-core-pool','justland','uniswap-v2','uniswap-v2','compound-v3','pendle','morpho-aave']

In [4]:
#TVL
## get most current tvl
def fetch_currenttvl_data(protocol):
    url = f"https://api.llama.fi/tvl/{protocol}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error occurred: {err}")
        
data_list = []

today_date = pd.Timestamp.now().date()

for protocol in protocols:
    data = fetch_currenttvl_data(protocol)
    data_list.append({'protocol': protocol, 'date': today_date, 'TVL': data})

most_current_tvl = pd.DataFrame(data_list)
print(most_current_tvl)

HTTP error occurred: 400 Client Error: Bad Request for url: https://api.llama.fi/tvl/dYdX
HTTP error occurred: 400 Client Error: Bad Request for url: https://api.llama.fi/tvl/dopex
               protocol        date           TVL
0                   gmx  2024-03-21  6.052521e+08
1          frax-finance  2024-03-21  1.345076e+09
2        convex-finance  2024-03-21  1.692631e+09
3         curve-finance  2024-03-21  2.716933e+09
4             synthetix  2024-03-21  8.897432e+08
5                  dYdX  2024-03-21           NaN
6         gains-network  2024-03-21  3.906124e+07
7         umami-finance  2024-03-21  6.968835e+06
8             LooksRare  2024-03-21  0.000000e+00
9                 Beefy  2024-03-21  2.540473e+08
10               ribbon  2024-03-21  1.542074e+07
11    redacted-protocol  2024-03-21  6.363800e+06
12                dopex  2024-03-21           NaN
13      kujira-protocol  2024-03-21  1.302596e+08
14  polynomial-protocol  2024-03-21  6.102228e+06
15                 

In [5]:
#trade volume

base_url = "https://api.llama.fi/summary/dexs"
volume_dfs = {}

for protocol in protocols:
    url = f"{base_url}/{protocol}?dataType=dailyVolume"
    response = requests.get(url)
    
    if response.status_code == 200:
        
        data = response.json()

        if 'totalDataChart' in data:
            total_data_chart = data['totalDataChart']

            df = pd.DataFrame(total_data_chart)

            df.columns = ['time', 'totalDataChart']

            df['time'] = pd.to_datetime(df['time'], unit='s')
            
            df.rename(columns={'time': 'date'}, inplace=True)
            
            df.rename(columns={'totalDataChart': 'volume'}, inplace=True)
            

            volume_dfs[protocol] = df
            
        else:
            print(f"'totalDataChart' not found in data for {protocol}")
    else:
        print(f"Failed to fetch data for {protocol}. Status code: {response.status_code}")

for protocol, df in volume_dfs.items():
    print(f"Data for {protocol}:\n", df.head())

Failed to fetch data for convex-finance. Status code: 404
Failed to fetch data for synthetix. Status code: 404
Failed to fetch data for dYdX. Status code: 502
Failed to fetch data for gains-network. Status code: 404
Failed to fetch data for umami-finance. Status code: 404
Failed to fetch data for LooksRare. Status code: 404
Failed to fetch data for Beefy. Status code: 404
Failed to fetch data for ribbon. Status code: 404
Failed to fetch data for redacted-protocol. Status code: 404
Failed to fetch data for dopex. Status code: 404
Failed to fetch data for kujira-protocol. Status code: 502
Failed to fetch data for polynomial-protocol. Status code: 502
Failed to fetch data for Lido. Status code: 404
Failed to fetch data for MakerDAO. Status code: 404
Failed to fetch data for aave. Status code: 502
Failed to fetch data for JustLend. Status code: 404
Failed to fetch data for compound-finance. Status code: 502
Failed to fetch data for venus. Status code: 502
Failed to fetch data for pendle. S

In [6]:
## Get most current volume 
most_current_data_with_protocol = []

for protocol, df in volume_dfs.items():
    most_current_row = df.loc[df['date'].idxmax()]
    most_current_row_dict = most_current_row.to_dict()
    most_current_row_dict['protocol'] = protocol  
    most_current_data_with_protocol.append(most_current_row_dict)

most_current_volume = pd.DataFrame(most_current_data_with_protocol)

columns_order = ['protocol'] + [col for col in most_current_volume.columns if col != 'protocol']
most_current_volume = most_current_volume[columns_order]

most_current_volume.reset_index(drop=True, inplace=True)

print(most_current_volume)

        protocol       date       volume
0            gmx 2024-03-20   49875910.0
1   frax-finance 2024-03-20    4826174.0
2  curve-finance 2024-03-20  374561212.0
3        uniswap 2024-03-20     163295.0
4    pancakeswap 2022-09-21     334055.0


In [7]:
#logos URL

protocols = ["gmx", "frax-finance", "convex-finance", "curve-finance", "synthetix", 'dYdX', 'gains-network', 'umami-finance', 'LooksRare', 'Beefy', 
             'ribbon', 'redacted-protocol', 'dopex', 'kujira-protocol', 'polynomial-protocol','Lido', 'MakerDAO', 'aave', 'JustLend', 'uniswap', 'compound-finance', 'pancakeswap', 'venus', 'pendle', 'morpho']

logos = {'protocol':["gmx", "frax-finance", "convex-finance", "curve-finance", "synthetix", 'dYdX', 'gains-network', 'umami-finance', 'LooksRare', 'Beefy', 
             'ribbon', 'redacted-protocol', 'dopex', 'kujira-protocol', 'polynomial-protocol','Lido', 'MakerDAO', 'aave', 'JustLend', 'uniswap', 'compound-finance', 'pancakeswap', 'venus', 'pendle', 'morpho'],
         
         'logos_urls':["https://altcoinsbox.com/wp-content/uploads/2023/03/gmx-logo.png",'https://cryptologos.cc/logos/frax-frax-logo.png','https://cryptologos.cc/logos/convex-finance-cvx-logo.png',
         'https://cryptologos.cc/logos/curve-dao-token-crv-logo.png','https://cryptologos.cc/logos/synthetix-snx-logo.png',
    'https://cryptologos.cc/logos/dydx-dydx-logo.png','https://gainsnetwork.io/images/logo_header.png','https://img.cryptorank.io/coins/umami_finance1660900013966.png','https://seeklogo.com/images/L/looksrare-logo-8A0876C037-seeklogo.com.png',
       'https://cryptologos.cc/logos/beefy-finance-bifi-logo.png','https://s2.coinmarketcap.com/static/img/coins/64x64/12387.png','https://icons.llamao.fi/icons/protocols/redacted-protocol','https://s1.coincarp.com/logo/1/dopex.png','https://s3.coinmarketcap.com/static/img/portraits/62e372f9ae5a2d740c0668d8.png','https://cdn-images-1.medium.com/max/280/1*4zhIPYqWmuFJ-znHnXg7rg@2x.png','https://cryptologos.cc/logos/lido-dao-ldo-logo.png' 
         ,'https://seeklogo.com/images/M/maker-mkr-logo-FAA728D102-seeklogo.com.png','https://cryptologos.cc/logos/aave-aave-logo.png','https://portal.justlend.org/static/media/logo.ac3cdcbc2778fb04940597b64d5ba949.svg', 
         'https://cryptologos.cc/logos/uniswap-uni-logo.png','https://cryptologos.cc/logos/compound-comp-logo.png','https://cryptologos.cc/logos/pancakeswap-cake-logo.png','https://research.binance.com/static/images/projects/venus/logo.png','https://s2.coinmarketcap.com/static/img/coins/200x200/9481.png','https://img.cryptorank.io/coins/morpho_labs1672304129665.png']}

logos_df = pd.DataFrame(logos)
logos_df

,protocol,logos_urls
0,gmx,https://altcoinsbox.com/wp-content/uploads/202...
1,frax-finance,https://cryptologos.cc/logos/frax-frax-logo.png
2,convex-finance,https://cryptologos.cc/logos/convex-finance-cv...
3,curve-finance,https://cryptologos.cc/logos/curve-dao-token-c...
4,synthetix,https://cryptologos.cc/logos/synthetix-snx-log...
5,dYdX,https://cryptologos.cc/logos/dydx-dydx-logo.png
6,gains-network,https://gainsnetwork.io/images/logo_header.png
7,umami-finance,https://img.cryptorank.io/coins/umami_finance1...
8,LooksRare,https://seeklogo.com/images/L/looksrare-logo-8...
9,Beefy,https://cryptologos.cc/logos/beefy-finance-bif...


In [8]:
#yield
base_url = "https://yields.llama.fi"

pools_endpoint = "/pools"

response = requests.get(base_url + pools_endpoint)

if response.status_code == 200:
    
    pools_data = response.json()

    pools = pools_data.get('data', [])

    pool_ids = {}

    for pool in pools:
        name = pool.get("project")
        pool_id = pool.get("pool")

        if name in protocols_pools:
            if name in protocols_pools:
                pool_ids[name] = pool_id
    print("Pool IDs for the specified protocols:")
    for protocol, pool_id in pool_ids.items():
        print(f"{protocol}: {pool_id}")

else:
    print(f"Failed to fetch pool data. Status code: {response.status_code}")

Pool IDs for the specified protocols:
lido: bf3a7f07-80a0-4d5b-a311-b0f06f650f83
aave-v3: 6ee54ffc-6ede-45a8-b35f-60aca8cc4176
makerdao: 404b111e-1619-47a9-a99b-431ebe53d0af
venus-core-pool: f02223ca-a69e-47df-8795-6657a8f829ee
compound-v3: 68e30785-53bf-49ef-892f-88b197c6bfe6
morpho-aave: bdc0b857-a1df-4714-82af-420093bc4b3f
pendle: daaae3ea-a090-4315-953c-b0ae3615264d
curve-dex: d51d5cf5-d1f3-4095-a3d5-f538fd0469f5
gmx-v1: c610c72f-40b7-40e8-8e20-fdd185d56ed8
uniswap-v2: 93d0bdcb-c52f-4fdc-aee7-c05bc560d3d9
convex-finance: c38f26ed-2bf7-44c6-a49b-2dc5d2fc8c95
gmx-v2: 0df3d94b-686a-4ec3-8161-43d82f283b5b
frax: 8cd2b4e9-1cf3-40d8-9981-454c33e7cf92
pancakeswap-amm: 7b3eabf6-5d72-4dba-8ed6-caf3441d2cdc
gains-network: c43627fb-3b95-436a-b93f-7c874d6c5f36
beefy: a6b4337c-82e9-47a2-bb84-bb82e395d586
looksrare: 134bd90b-be1b-496d-9fea-8b8a50fc8dbf
ribbon: 0c01766a-6005-4968-abd6-2129678cf1e0
umami-finance: e07477a3-fc27-4b57-8b22-685c07f2abec


In [9]:
base_url = "https://yields.llama.fi/chart"

protocol_dfs = {}

for protocol, pool_id in pool_ids.items():

    url = f"{base_url}/{pool_id}"

    response = requests.get(url)

    if response.status_code == 200:

        data = response.json()
        df = pd.DataFrame(data)

        protocol_dfs[protocol] = df

        print(f"Yield data for {protocol} has been successfully retrieved and stored in a DataFrame.")
        
    else:
        print(f"Failed to fetch yield data for {protocol}. Status code: {response.status_code}")

Yield data for lido has been successfully retrieved and stored in a DataFrame.
Yield data for aave-v3 has been successfully retrieved and stored in a DataFrame.
Yield data for makerdao has been successfully retrieved and stored in a DataFrame.
Yield data for venus-core-pool has been successfully retrieved and stored in a DataFrame.
Yield data for compound-v3 has been successfully retrieved and stored in a DataFrame.
Yield data for morpho-aave has been successfully retrieved and stored in a DataFrame.
Yield data for pendle has been successfully retrieved and stored in a DataFrame.
Yield data for curve-dex has been successfully retrieved and stored in a DataFrame.
Yield data for gmx-v1 has been successfully retrieved and stored in a DataFrame.
Yield data for uniswap-v2 has been successfully retrieved and stored in a DataFrame.
Yield data for convex-finance has been successfully retrieved and stored in a DataFrame.
Yield data for gmx-v2 has been successfully retrieved and stored in a Data

In [10]:
yield_dfs = {}

for protocol, df in protocol_dfs.items():
    
    df['timestamp'] = df['data'].apply(lambda x: datetime.strptime(x['timestamp'], "%Y-%m-%dT%H:%M:%S.%fZ"))
    
    df['APY'] = df['data'].apply(lambda x: x['apy'])

    df_cleaned = df.drop(columns=['data', 'status'])

    yield_dfs[protocol] = df_cleaned

    print(f"Data for {protocol} cleaned and stored.")

for protocol, df in yield_dfs.items():

    df['timestamp'] = df['timestamp'].dt.date
    
    df.rename(columns={'timestamp': 'date'}, inplace=True)

    yield_dfs[protocol] = df

    print(f"Data for {protocol} updated.")

Data for lido cleaned and stored.
Data for aave-v3 cleaned and stored.
Data for makerdao cleaned and stored.
Data for venus-core-pool cleaned and stored.
Data for compound-v3 cleaned and stored.
Data for morpho-aave cleaned and stored.
Data for pendle cleaned and stored.
Data for curve-dex cleaned and stored.
Data for gmx-v1 cleaned and stored.
Data for uniswap-v2 cleaned and stored.
Data for convex-finance cleaned and stored.
Data for gmx-v2 cleaned and stored.
Data for frax cleaned and stored.
Data for pancakeswap-amm cleaned and stored.
Data for gains-network cleaned and stored.
Data for beefy cleaned and stored.
Data for looksrare cleaned and stored.
Data for ribbon cleaned and stored.
Data for umami-finance cleaned and stored.
Data for lido updated.
Data for aave-v3 updated.
Data for makerdao updated.
Data for venus-core-pool updated.
Data for compound-v3 updated.
Data for morpho-aave updated.
Data for pendle updated.
Data for curve-dex updated.
Data for gmx-v1 updated.
Data for u

In [11]:
# Sample dictionary of DataFrames

# Initialize lists to store key and last row data
keys = []
last_rows = []

# Iterate over dictionary items
for protocol, df in yield_dfs.items():
    
    most_current_row = df.loc[df['date'].idxmax()]
    most_current_row_dict = most_current_row.to_dict()
    most_current_row_dict['protocol'] = protocol  
    last_rows.append(most_current_row_dict)

# Create a new DataFrame from extracted data
most_current_apy = pd.DataFrame(last_rows)
columns_order = ['protocol'] + [col for col in most_current_apy.columns if col != 'protocol']
most_current_apy = most_current_apy[columns_order]
most_current_apy.reset_index(drop=True, inplace=True)

# Display the new DataFrame
print(most_current_apy)

           protocol        date       APY
0              lido  2024-03-21   4.27000
1           aave-v3  2024-03-21   2.45165
2          makerdao  2024-03-21   0.00000
3   venus-core-pool  2024-03-21  24.67198
4       compound-v3  2024-03-21   0.00000
5       morpho-aave  2024-03-21  10.76984
6            pendle  2024-03-21   0.31203
7         curve-dex  2024-03-21   0.00000
8            gmx-v1  2024-03-21   6.36844
9        uniswap-v2  2024-03-21   0.00000
10   convex-finance  2024-03-21   1.60000
11           gmx-v2  2024-03-21   0.36911
12             frax  2024-03-21   0.00000
13  pancakeswap-amm  2024-03-21  11.68000
14    gains-network  2024-03-21  34.91543
15            beefy  2024-03-21  21.23462
16        looksrare  2024-03-21  51.23991
17           ribbon  2024-03-21   0.00148
18    umami-finance  2024-03-21  31.75000


In [12]:
#fix names
most_current_tvl['protocol'] = most_current_tvl['protocol'].str.lower()
logos_df['protocol'] = logos_df['protocol'].str.lower()

most_current_apy.loc[1, 'protocol'] = 'aave'
most_current_apy.loc[3, 'protocol'] = 'venus'
most_current_apy.loc[4, 'protocol'] = 'compound-finance'
most_current_apy.loc[5, 'protocol'] = 'morpho'
most_current_apy.loc[7, 'protocol'] = 'gmx'
most_current_apy.loc[12,'protocol'] = "frax-finance"
most_current_apy.loc[13,'protocol'] = "pancakeswap"

In [13]:
#merging them into one data frame.
df_list = [most_current_tvl,most_current_volume,most_current_apy]

merged_df = pd.merge(most_current_tvl[['protocol', 'TVL']], most_current_volume[['protocol', 'volume']], on='protocol',how="left")
merged_df = pd.merge(merged_df, most_current_apy[['protocol', 'APY']],on = 'protocol',how ="left")
merged_df = pd.merge(merged_df, logos_df, on = 'protocol',how ="left")

In [14]:
merged_df

,protocol,TVL,volume,APY,logos_urls
0,gmx,6.052521e+08,49875910.0,0.00000,https://altcoinsbox.com/wp-content/uploads/202...
1,frax-finance,1.345076e+09,4826174.0,0.00000,https://cryptologos.cc/logos/frax-frax-logo.png
2,convex-finance,1.692631e+09,NaN,1.60000,https://cryptologos.cc/logos/convex-finance-cv...
3,curve-finance,2.716933e+09,374561212.0,NaN,https://cryptologos.cc/logos/curve-dao-token-c...
4,synthetix,8.897432e+08,NaN,NaN,https://cryptologos.cc/logos/synthetix-snx-log...
5,dydx,NaN,NaN,NaN,https://cryptologos.cc/logos/dydx-dydx-logo.png
6,gains-network,3.906124e+07,NaN,34.91543,https://gainsnetwork.io/images/logo_header.png
7,umami-finance,6.968835e+06,NaN,31.75000,https://img.cryptorank.io/coins/umami_finance1...
8,looksrare,0.000000e+00,NaN,51.23991,https://seeklogo.com/images/L/looksrare-logo-8...
9,beefy,2.540473e+08,NaN,21.23462,https://cryptologos.cc/logos/beefy-finance-bif...


In [15]:
merged_df

,protocol,TVL,volume,APY,logos_urls
0,gmx,6.052521e+08,49875910.0,0.00000,https://altcoinsbox.com/wp-content/uploads/202...
1,frax-finance,1.345076e+09,4826174.0,0.00000,https://cryptologos.cc/logos/frax-frax-logo.png
2,convex-finance,1.692631e+09,NaN,1.60000,https://cryptologos.cc/logos/convex-finance-cv...
3,curve-finance,2.716933e+09,374561212.0,NaN,https://cryptologos.cc/logos/curve-dao-token-c...
4,synthetix,8.897432e+08,NaN,NaN,https://cryptologos.cc/logos/synthetix-snx-log...
5,dydx,NaN,NaN,NaN,https://cryptologos.cc/logos/dydx-dydx-logo.png
6,gains-network,3.906124e+07,NaN,34.91543,https://gainsnetwork.io/images/logo_header.png
7,umami-finance,6.968835e+06,NaN,31.75000,https://img.cryptorank.io/coins/umami_finance1...
8,looksrare,0.000000e+00,NaN,51.23991,https://seeklogo.com/images/L/looksrare-logo-8...
9,beefy,2.540473e+08,NaN,21.23462,https://cryptologos.cc/logos/beefy-finance-bif...


In [29]:
desired_column_order = ['logos_urls','protocol', 'TVL', 'volume','APY']
merged_df = merged_df[desired_column_order]
merged_df.loc[:, 'APY'] = ['14.64%','3.99% - 4.24%','20.68%','0.00% - 50.00%','','33.06%','27.67%','27.34%','51.10%','4.48% - 18.56%','8.90%','3.87% - 9.75%','23.61% - 50.35%','5.00% – 10.00%','1.00% - 3.00%','4.11%','1.00% - 15.00%','','','0.00% - 100.00%','6.60% - 18.19%','0.00% - 100.00%','','10.50% - 32.90%','2.45%']
#Calculated by the transformer
merged_df.loc[:, 'Risk Level'] = ['Low','Low','Low','Low','Low','Low','Low','Low','Low','Low','Low','Low','Invalid Value','Invalid Value','Low','Medium High','Low','Medium','Medium','Low','Low','Low','Low','Low','Low']
df_rounded = merged_df.round(2)
df_rounded


,logos_urls,protocol,TVL,volume,APY,Risk Level
0,https://altcoinsbox.com/wp-content/uploads/202...,gmx,6.052521e+08,49875910.0,14.64%,Low
1,https://cryptologos.cc/logos/frax-frax-logo.png,frax-finance,1.345076e+09,4826174.0,3.99% - 4.24%,Low
2,https://cryptologos.cc/logos/convex-finance-cv...,convex-finance,1.692631e+09,NaN,20.68%,Low
3,https://cryptologos.cc/logos/curve-dao-token-c...,curve-finance,2.716933e+09,374561212.0,0.00% - 50.00%,Low
4,https://cryptologos.cc/logos/synthetix-snx-log...,synthetix,8.897432e+08,NaN,,Low
5,https://cryptologos.cc/logos/dydx-dydx-logo.png,dydx,NaN,NaN,33.06%,Low
6,https://gainsnetwork.io/images/logo_header.png,gains-network,3.906124e+07,NaN,27.67%,Low
7,https://img.cryptorank.io/coins/umami_finance1...,umami-finance,6.968835e+06,NaN,27.34%,Low
8,https://seeklogo.com/images/L/looksrare-logo-8...,looksrare,0.000000e+00,NaN,51.10%,Low
9,https://cryptologos.cc/logos/beefy-finance-bif...,beefy,2.540473e+08,NaN,4.48% - 18.56%,Low


In [30]:
df_rounded.to_csv('display_dataframe.csv')